# Variable-Duration Data from Horsager et al. 2009

In [1]:
from os import environ
import pandas as pd
import numpy as np

In [2]:
def open_excel(filename, idx_sheet=0):
    xl = pd.ExcelFile(filename)
    df = xl.parse(xl.sheet_names[idx_sheet])
    assert 'Subject' in df.columns
    assert 'Electrode' in df.columns
    assert 'pdur' in df.columns
    assert 'n_pulses' in df.columns
    assert 'type' in df.columns
    assert 'data' in df.columns
    df.drop(df.index, inplace=True)
    return df

In [3]:
def load_row(df, subject, electrode, pdur, n_pulses, dtype='truth'):
    rows = (df['Subject'] == subject) & (df['Electrode'] == electrode) & \
           (df['type'] == dtype) & (df['pdur'] == pdur) & \
           (df['n_pulses'] == n_pulses)
    if not np.any(rows):
        return None, None
    elif np.count_nonzero(rows) > 1:
        print('More than one row found:')
        print(df.loc[rows, :])
        return None, None

    # Data points: [all pdurs, all amps]
    data = np.array(df.loc[rows, 'data':])
    data = data.reshape((2, -1))
    return data[0, :], data[1, :]

In [4]:
def append_row(df, subject, electrode, theta, pdur, n_pulses, dtype, freqs, amps):
    # Add all data into a single np.array
    arr = np.concatenate((np.array([subject, electrode, theta, pdur, n_pulses, dtype]), freqs, amps))
    
    # Make a PD series from it
    newrow = pd.Series(arr, index=df.columns)
    return df.append(newrow, ignore_index=True)

In [5]:
df = open_excel(environ['SECOND_SIGHT_DATA'] + '/horsager2009/variable-duration.xlsx')
df

,Subject,Electrode,theta,pdur,n_pulses,type,data,Unnamed: 5,Unnamed: 6,Unnamed: 7,...,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21


## Fig. 5

In [6]:
from scipy.interpolate import interp1d

truth = np.array([3.07744, 177.21735, 6.83354, 161.49903,
                  10.00000, 152.50483, 19.55864, 151.48269,
                  48.42163, 138.42371, 94.70610, 124.19403,
                  178.63580, 134.86406, 426.50218, 125.19536,
                  761.88560, 133.96969]).reshape((-1, 2))
x_axis = truth[:, 0]
df = append_row(df, 'S05', 'C3', 65.0, 0.075, 2, 'truth', truth[:, 0], truth[:, 1])

pred = np.array([3.07744, 162.12301, 6.95856, 163.76616,
                 10.18296, 154.01723, 19.20722, 145.44198,
                 48.42163, 139.93315, 100.00000, 137.03313,
                 178.63580, 136.75085, 426.50218, 138.02555,
                 734.75362, 141.13356]).reshape((-1, 2))
df = append_row(df, 'S05', 'C3', 65.0, 0.075, 2, 'pred', pred[:, 0], pred[:, 1])

truth = np.array([3.10786, 83.43185, 7.36733, 88.12425,
                               10.52728, 84.33655, 21.42748, 73.99276,
                               49.64420, 78.38359, 99.68801, 76.04343,
                               195.71153, 73.70929, 460.09425, 71.01930,
                               868.52368, 73.92642, 1254.03951, 79.36671,
                               1953.04253, 71.86369, 2802.38075, 71.76719,
                               3521.62464, 74.16767]).reshape((-1, 2))
f = interp1d(truth[:, 0], truth[:, 1], fill_value='extrapolate')
df = append_row(df, 'S06', 'A1', 7.5, 0.075, 2, 'truth', x_axis, f(x_axis))

pred = np.array([3.11434, 85.27744, 7.33670, 84.43305,
                                 20.00395, 73.08806, 50.28605, 69.76478,
                                 101.22259, 69.57780, 194.83017, 69.71049,
                                 459.13680, 69.17370, 826.74262, 70.24729,
                                 1299.24542, 70.74186, 1827.72959, 73.11218,
                                 2567.61293, 74.25211, 3224.36285, 76.03739]).reshape((-1, 2))
f = interp1d(pred[:, 0], pred[:, 1], fill_value='extrapolate')
df = append_row(df, 'S06', 'A1', 7.5, 0.075, 2, 'pred', x_axis, f(x_axis))


truth = np.array([3.05386, 197.09596, 7.39072, 191.44571,
                               10.23531, 173.69792, 20.56512, 137.45265,
                               47.50810, 95.83333, 100.00000, 90.86963,
                               191.79103, 74.90530, 443.06215, 80.16098,
                               792.48290, 85.24306, 1176.81195, 77.69886,
                               1788.64953, 81.10795, 2477.07636, 86.79766,
                               3125.71585, 101.79924]).reshape((-1, 2))
f = interp1d(truth[:, 0], truth[:, 1], fill_value='extrapolate')
df = append_row(df, 'S05', 'C3', 51.3, 0.075, 15, 'truth', x_axis, f(x_axis))

pred = np.array([3.19927, 161.19003, 7.05480, 157.03914,
                                 10.00000, 151.02588, 19.63041, 133.51484,
                                 48.62602, 105.22412, 100.00000, 93.21338,
                                 191.79103, 87.40530, 443.06215, 83.28598,
                                 792.48290, 91.49306, 1149.75700, 97.99558,
                                 1788.64953, 107.67045, 2364.48941, 117.23485,
                                 2983.64724, 128.33018]).reshape((-1, 2))
f = interp1d(pred[:, 0], pred[:, 1], fill_value='extrapolate')
df = append_row(df, 'S05', 'C3', 51.3, 0.075, 15, 'pred', x_axis, f(x_axis))

truth = np.array([3.52346, 76.36364, 8.04368, 74.84848,
                               11.33824, 73.93939, 23.08518, 72.72727,
                               53.92848, 61.21212, 109.03747, 56.96970,
                               201.34433, 43.33333, 482.99140, 53.33333,
                               881.35630, 54.54545, 1263.32518, 50.90909,
                               2043.16307, 49.69697, 2834.15920, 41.81818,
                               3469.78872, 49.69697]).reshape((-1, 2))
f = interp1d(truth[:, 0], truth[:, 1], fill_value='extrapolate')
df = append_row(df, 'S06', 'A1', 5.1, 0.075, 15, 'truth', x_axis, f(x_axis))

pred = np.array([3.55805, 80.60606, 8.49951, 78.78788,
                                 11.10336, 74.84848, 22.79706, 67.27273,
                                 51.68129, 52.72727, 109.26595, 47.87879,
                                 206.46597, 44.24242, 471.66795, 43.03030,
                                 861.89537, 44.84848, 1369.82998, 46.06061,
                                 2010.63544, 52.72727, 2687.78137, 58.78788,
                                 3582.96383, 63.63636]).reshape((-1, 2))
f = interp1d(pred[:, 0], pred[:, 1], fill_value='extrapolate')
df = append_row(df, 'S06', 'A1', 5.1, 0.075, 15, 'pred', x_axis, f(x_axis))

## Fig. S2.3

In [7]:
# truth = np.array([5.00000, 112.83784, 15.14525, 92.56757,
#                   45.00000, 70.27027, 74.99495, 76.35135,
#                   135.00000, 61.48649, 224.98486, 61.48649]).reshape((-1, 2))
# df = append_row(df, 'S05', 'B3', 21.1, 0.075, 'truth', truth[:, 0], truth[:, 1])

# truth = np.array([4.75490, 214.59597, 14.72461, 191.10352,
#                   45.31173, 131.81568, 76.40481, 105.30261,
#                   137.79166, 93.52883, 234.09605, 90.46412]).reshape((-1, 2))
# df = append_row(df, 'S05', 'C2', 10.5, 0.075, 'truth', truth[:, 0], truth[:, 1])

# truth = np.array([5.04958, 202.38095, 15.11228, 177.38095,
#                   45.25887, 159.52381, 76.02583, 141.66667,
#                   135.40329, 130.95238, 225.42409, 120.23810]).reshape((-1, 2))
# df = append_row(df, 'S06', 'B1', 2500.0, 0.075, 'truth', truth[:, 0], truth[:, 1])

# truth = np.array([5.26202, 269.35835, 15.73428, 252.16150,
#                   45.90965, 156.76998, 76.21561, 159.77433,
#                   135.84947, 134.85590, 225.12285, 128.48015]).reshape((-1, 2))
# df = append_row(df, 'S06', 'C2', 230.0, 0.075, 'truth', truth[:, 0], truth[:, 1])

## Fig. S2.4

In [8]:
# truth = np.array([5.05160, 29.55563, 15.00000, 23.71010,
#                   44.54033, 23.90231, 74.42302, 22.40522,
#                   132.25607, 19.37753, 218.73103, 19.01605]).reshape((-1, 2))
# df = append_row(df, 'S05', 'B3', 147.0, 0.975, 'truth', truth[:, 0], truth[:, 1])

# truth = np.array([4.94796, 24.84375, 15.00000, 23.12500,
#                   45.95159, 22.50000, 76.72857, 20.93750,
#                   139.30471, 20.93750, 232.60679, 20.31250]).reshape((-1, 2))
# df = append_row(df, 'S05', 'C2', 54.0, 0.975, 'truth', truth[:, 0], truth[:, 1])

# truth = np.array([5.00000, 23.00000, 14.84678, 25.16667,
#                   44.08536, 25.16667, 72.91034, 22.50000,
#                   132.25607, 25.83333, 218.73103, 22.83333]).reshape((-1, 2))
# df = append_row(df, 'S06', 'B1', 2900.0, 0.975, 'truth', truth[:, 0], truth[:, 1])

# truth = np.array([4.77221, 34.34783, 14.62270, 34.13043,
#                   44.51137, 30.21739, 74.30683, 30.43478,
#                   135.43058, 26.73913, 226.54974, 25.00000]).reshape((-1, 2))
# df = append_row(df, 'S06', 'C2', 900.0, 0.975, 'truth', truth[:, 0], truth[:, 1])

## Fig. S3.3

In [9]:
# truth = np.array([4.83173, 106.56324, 14.27496, 91.78543,
#                   42.65204, 69.01394, 70.97810, 59.45306,
#                   125.93917, 56.69734, 209.36605, 51.44360]).reshape((-1, 2))
# df = append_row(df, 'S05', 'A1', 10.5, 0.075, 'truth', truth[:, 0], truth[:, 1])

# truth = np.array([5.03690, 179.04947, 15.18142, 127.50746,
#                   45.88949, 105.86506, 78.59943, 89.87519,
#                   140.05534, 84.23171, 233.28153, 78.52492]).reshape((-1, 2))
# df = append_row(df, 'S05', 'C4', 49.5, 0.075, 'truth', truth[:, 0], truth[:, 1])

# truth = np.array([4.88165, 251.47059, 14.70626, 207.35294,
#                   44.32818, 157.35294, 73.40122, 132.35294,
#                   131.93299, 141.17647, 218.34075, 122.05882]).reshape((-1, 2))
# df = append_row(df, 'S06', 'B2', 216.0, 0.075, 'truth', truth[:, 0], truth[:, 1])

# truth = np.array([4.81146, 405.88235, 14.80072, 281.17647,
#                   43.60594, 260.00000, 71.79073, 222.35294,
#                   128.81898, 182.35294, 216.41555, 121.17647]).reshape((-1, 2))
# df = append_row(df, 'S06', 'D1', 188.0, 0.075, 'truth', truth[:, 0], truth[:, 1])

## Fig. S3.4

In [10]:
# truth = np.array([5.02108, 24.36620, 15.16171, 21.12676,
#                   46.26343, 18.73239, 77.70989, 16.90141,
#                   139.67851, 15.35211, 232.56079, 14.36620]).reshape((-1, 2))
# df = append_row(df, 'S05', 'A1', 154.0, 0.975, 'truth', truth[:, 0], truth[:, 1])

# truth = np.array([5.04799, 18.29657, 15.00000, 15.35011,
#                   45.00000, 18.08696, 75.37929, 17.61648,
#                   133.71647, 16.29840, 226.13788, 15.51121]).reshape((-1, 2))
# df = append_row(df, 'S05', 'C4', 141.8, 0.975, 'truth', truth[:, 0], truth[:, 1])

# truth = np.array([5.00000, 30.54545, 15.14525, 28.18182,
#                   45.87574, 28.54545, 77.94229, 28.72727,
#                   140.30555, 26.54545, 236.09113, 24.18182]).reshape((-1, 2))
# df = append_row(df, 'S06', 'B2', 546.0, 0.975, 'truth', truth[:, 0], truth[:, 1])

# truth = np.array([5.04799, 30.89286, 15.00000, 29.28571,
#                   44.57216, 29.64286, 75.37929, 30.53571,
#                   133.71647, 27.50000, 223.98784, 23.57143]).reshape((-1, 2))
# df = append_row(df, 'S06', 'D1', 588.0, 0.975, 'truth', truth[:, 0], truth[:, 1])

In [11]:
df

,Subject,Electrode,theta,pdur,n_pulses,type,data,Unnamed: 5,Unnamed: 6,Unnamed: 7,...,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21
0,S05,C3,65.0,0.075,2,truth,3.07744,6.83354,10.0,19.55864,...,761.8856,177.21735,161.49903,152.50483,151.48269,138.42371,124.19403,134.86406,125.19536,133.96969
1,S05,C3,65.0,0.075,2,pred,3.07744,6.95856,10.18296,19.20722,...,734.75362,162.12301,163.76616,154.01723,145.44198,139.93315,137.03313,136.75085,138.02555,141.13356
2,S06,A1,7.5,0.075,2,truth,3.07744,6.83354,10.0,19.55864,...,761.8856,83.3983381292743,87.53620590155583,84.96857849918511,75.76620346925745,78.19334474707549,76.27639520599851,74.12436689389224,71.36108607580442,73.16739121226156
3,S06,A1,7.5,0.075,2,pred,3.07744,6.83354,10.0,19.55864,...,761.8856,85.28481928338654,84.53367222842202,82.0477562806055,73.48688669852572,69.96938898344567,69.601721006417,69.68753426334598,69.23997884313685,70.05787556662189
4,S05,C3,51.3,0.075,15,truth,3.07744,6.83354,10.0,19.55864,...,761.8856,197.06523894951647,192.1716283591354,175.16605159889474,140.98419020738038,95.74694545569888,91.37022783376102,77.19326248332654,79.81460552158958,84.79804370836878
5,S05,C3,51.3,0.075,15,pred,3.07744,6.83354,10.0,19.55864,...,761.8856,161.3211930122707,157.27735003115006,151.02588,133.6453398791121,105.42354123172439,94.45104265502498,88.2376975475425,83.5574629130399,90.77440065053663
6,S06,A1,5.1,0.075,15,truth,3.07744,6.83354,10.0,19.55864,...,761.8856,76.5131441531607,75.25411417762852,74.30865952357826,73.09115963124015,63.2680674432859,58.072962656699026,46.6880310182786,51.32765601943986,54.1819319669504
7,S06,A1,5.1,0.075,15,pred,3.07744,6.83354,10.0,19.55864,...,761.8856,80.78289751154517,79.40086347747427,76.51776831691534,69.37073664588624,54.36876275032085,49.10469185525103,45.28358023106683,43.23673259576116,44.382506174740364


In [12]:
writer = pd.ExcelWriter(environ['SECOND_SIGHT_DATA'] + '/horsager2009/variable-duration.xlsx')

In [13]:
df.to_excel(writer,'Sheet1')

In [14]:
writer.save()